<a href="https://colab.research.google.com/github/jvargh81/wallmart/blob/master/wallmart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.exceptions import NotFittedError
from sklearn.model_selection import RandomizedSearchCV
from functools import partial
from sklearn.preprocessing import StandardScaler
from google.colab import drive
drive.mount('/content/drive')
import warnings  
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
wallmart = pd.read_csv("/content/drive/My Drive/train (1).csv")
wallmart.head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
0,999,5,Friday,6.811315e+10,-1,FINANCIAL SERVICES,1000.0
1,30,7,Friday,6.053882e+10,1,SHOES,8931.0
2,30,7,Friday,7.410811e+09,1,PERSONAL CARE,4504.0
3,26,8,Friday,2.238404e+09,2,PAINT AND ACCESSORIES,3565.0
4,26,8,Friday,2.006614e+09,2,PAINT AND ACCESSORIES,1017.0


In [4]:
#replacing nan from Upc
wallmart["Upc"].fillna(0, inplace=True)
wallmart["DepartmentDescription"].unique()

array(['FINANCIAL SERVICES', 'SHOES', 'PERSONAL CARE',
       'PAINT AND ACCESSORIES', 'DSD GROCERY', 'MEAT - FRESH & FROZEN',
       'DAIRY', 'PETS AND SUPPLIES', 'HOUSEHOLD CHEMICALS/SUPP', nan,
       'IMPULSE MERCHANDISE', 'PRODUCE', 'CANDY, TOBACCO, COOKIES',
       'GROCERY DRY GOODS', 'BOYS WEAR', 'FABRICS AND CRAFTS',
       'JEWELRY AND SUNGLASSES', 'MENS WEAR', 'ACCESSORIES',
       'HOME MANAGEMENT', 'FROZEN FOODS', 'SERVICE DELI',
       'INFANT CONSUMABLE HARDLINES', 'PRE PACKED DELI', 'COOK AND DINE',
       'PHARMACY OTC', 'LADIESWEAR', 'COMM BREAD', 'BAKERY',
       'HOUSEHOLD PAPER GOODS', 'CELEBRATION', 'HARDWARE', 'BEAUTY',
       'AUTOMOTIVE', 'BOOKS AND MAGAZINES', 'SEAFOOD', 'OFFICE SUPPLIES',
       'LAWN AND GARDEN', 'SHEER HOSIERY', 'WIRELESS', 'BEDDING',
       'BATH AND SHOWER', 'HORTICULTURE AND ACCESS', 'HOME DECOR', 'TOYS',
       'INFANT APPAREL', 'LADIES SOCKS', 'PLUS AND MATERNITY',
       'ELECTRONICS', 'GIRLS WEAR, 4-6X  AND 7-14', 'BRAS & SHAPEWEAR',

In [0]:
def char2numeric(data,column) :
    for i in column :
        for j in range(0,len(data[i].unique())):
            data = data.replace(data[i].unique()[j],j)
        
    return data;

In [6]:
wallmart['TripType'] = wallmart['TripType'].astype('category')
wallmart['VisitNumber'] = wallmart['VisitNumber'].astype('category')
wallmart['Weekday'] = wallmart['Weekday'].astype('category')
wallmart['Upc'] = wallmart['Upc'].astype('category')
wallmart['ScanCount'] = wallmart['ScanCount'].astype('category')
wallmart["DepartmentDescription"] = wallmart["DepartmentDescription"].astype('category')
wallmart['FinelineNumber'] = wallmart['FinelineNumber'].astype('category')
wallmart["TripType_cat"] = wallmart["TripType"].cat.codes
wallmart["VisitNumber_cat"] = wallmart["VisitNumber"].cat.codes
wallmart["Weekday_cat"] = wallmart["Weekday"].cat.codes
wallmart["Upc_cat"] = wallmart["Upc"].cat.codes
wallmart["ScanCount_cat"] = wallmart["ScanCount"].cat.codes
wallmart['Department_cat'] = wallmart['DepartmentDescription'].cat.codes
wallmart["FinelineNumber_cat"] = wallmart["FinelineNumber"].cat.codes
print("Dataframe dimension:",wallmart.shape)
print("Datatypes :",wallmart.dtypes)
wallmart = wallmart[['TripType_cat','VisitNumber_cat','Weekday_cat','Upc_cat',
       'ScanCount_cat','Department_cat','FinelineNumber_cat']]

Dataframe dimension: (647054, 14)
Datatypes : TripType                 category
VisitNumber              category
Weekday                  category
Upc                      category
ScanCount                category
DepartmentDescription    category
FinelineNumber           category
TripType_cat                 int8
VisitNumber_cat             int32
Weekday_cat                  int8
Upc_cat                     int32
ScanCount_cat                int8
Department_cat               int8
FinelineNumber_cat          int16
dtype: object


In [7]:
#Looking at the new data after numeric data added.
wallmart.head()

,TripType_cat,VisitNumber_cat,Weekday_cat,Upc_cat,ScanCount_cat,Department_cat,FinelineNumber_cat
0,37,0,0,66661,9,20,746
1,22,1,0,58982,10,62,4885
2,22,1,0,41387,10,50,2852
3,18,2,0,9252,11,49,2322
4,18,2,0,7860,11,49,763


In [0]:
# Convert into features and class

#Features
X = wallmart.values[:,1:7]

# Label/Class
Y = wallmart["TripType_cat"]

# Spliting into train and test data
x_data_train,x_data_test,y_data_train,y_data_test = train_test_split(X,Y,test_size = 0.1,random_state = 100)


In [0]:
scaler = StandardScaler()
x_data_train = scaler.fit_transform(x_data_train)
scaler = StandardScaler()
x_data_test = scaler.fit_transform(x_data_test)
y_data_train = np.array(y_data_train)
y_data_test = np.array(y_data_test)

In [0]:
n_inputs = 6
n_outputs = 45
batch_norm_momentum = 0.9
n_hidden1 = 300
n_hidden2 = 100 
learning_rate = 0.1

In [0]:
he_init = tf.variance_scaling_initializer()
class DNNClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, n_hidden_layers=5, n_neurons=100, optimizer_class=tf.train.AdamOptimizer,
                 learning_rate=0.01, batch_size=20, activation=tf.nn.relu, initializer=he_init,
                 batch_norm_momentum=None, dropout_rate=None, random_state=None):
        """Initialize the DNNClassifier by simply storing all the hyperparameters."""
        self.n_hidden_layers = n_hidden_layers
        self.n_neurons = n_neurons
        self.optimizer_class = optimizer_class
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.activation = activation
        self.initializer = initializer
        self.batch_norm_momentum = batch_norm_momentum
        self.dropout_rate = dropout_rate
        self.random_state = random_state
        self._session = None

    def _dnn(self, inputs):
        """Build the hidden layers, with support for batch normalization and dropout."""
        for layer in range(self.n_hidden_layers):
            if self.dropout_rate:
                inputs = tf.layers.dropout(inputs, self.dropout_rate, training=self._training)
            inputs = tf.layers.dense(inputs, self.n_neurons,
                                     kernel_initializer=self.initializer,
                                     name="hidden%d" % (layer + 1))
            if self.batch_norm_momentum:
                inputs = tf.layers.batch_normalization(inputs, momentum=self.batch_norm_momentum,
                                                       training=self._training)
            inputs = self.activation(inputs, name="hidden%d_out" % (layer + 1))
        return inputs

    def _build_graph(self, n_inputs, n_outputs):
        """Build the same model as earlier"""
        if self.random_state is not None:
            tf.set_random_seed(self.random_state)
            np.random.seed(self.random_state)

        X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
        y = tf.placeholder(tf.int32, shape=(None), name="y")

        if self.batch_norm_momentum or self.dropout_rate:
            self._training = tf.placeholder_with_default(False, shape=(), name='training')
        else:
            self._training = None

        dnn_outputs = self._dnn(X)

        logits = tf.layers.dense(dnn_outputs, n_outputs, kernel_initializer=he_init, name="logits")
        Y_proba = tf.nn.softmax(logits, name="Y_proba")

        xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                                  logits=logits)
        loss = tf.reduce_mean(xentropy, name="loss")

        optimizer = self.optimizer_class(learning_rate=self.learning_rate)
        training_op = optimizer.minimize(loss)

        correct = tf.nn.in_top_k(logits, y, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

        init = tf.global_variables_initializer()
        saver = tf.train.Saver()

        # Make the important operations available easily through instance variables
        self._X, self._y = X, y
        self._Y_proba, self._loss = Y_proba, loss
        self._training_op, self._accuracy = training_op, accuracy
        self._init, self._saver = init, saver

    def close_session(self):
        if self._session:
            self._session.close()

    def _get_model_params(self):
        """Get all variable values (used for early stopping, faster than saving to disk)"""
        with self._graph.as_default():
            gvars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
        return {gvar.op.name: value for gvar, value in zip(gvars, self._session.run(gvars))}

    def _restore_model_params(self, model_params):
        """Set all variables to the given values (for early stopping, faster than loading from disk)"""
        gvar_names = list(model_params.keys())
        assign_ops = {gvar_name: self._graph.get_operation_by_name(gvar_name + "/Assign")
                      for gvar_name in gvar_names}
        init_values = {gvar_name: assign_op.inputs[1] for gvar_name, assign_op in assign_ops.items()}
        feed_dict = {init_values[gvar_name]: model_params[gvar_name] for gvar_name in gvar_names}
        self._session.run(assign_ops, feed_dict=feed_dict)

    def fit(self, X, y, n_epochs=100, X_valid=None, y_valid=None):
        """Fit the model to the training set. If X_valid and y_valid are provided, use early stopping."""
        self.close_session()

        # infer n_inputs and n_outputs from the training set.
        n_inputs = X.shape[1]
        self.classes_ = np.unique(y)
        n_outputs = len(self.classes_)
        
        # Translate the labels vector to a vector of sorted class indices, containing
        # integers from 0 to n_outputs - 1.
        # For example, if y is equal to [8, 8, 9, 5, 7, 6, 6, 6], then the sorted class
        # labels (self.classes_) will be equal to [5, 6, 7, 8, 9], and the labels vector
        # will be translated to [3, 3, 4, 0, 2, 1, 1, 1]
        self.class_to_index_ = {label: index
                                for index, label in enumerate(self.classes_)}
        y = np.array([self.class_to_index_[label]
                      for label in y], dtype=np.int32)
        
        self._graph = tf.Graph()
        with self._graph.as_default():
            self._build_graph(n_inputs, n_outputs)
            # extra ops for batch normalization
            extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

        # needed in case of early stopping
        max_checks_without_progress = 20
        checks_without_progress = 0
        best_loss = np.infty
        best_params = None
        
        # Now train the model!
        self._session = tf.Session(graph=self._graph)
        with self._session.as_default() as sess:
            self._init.run()
            for epoch in range(n_epochs):
                rnd_idx = np.random.permutation(len(X))
                for rnd_indices in np.array_split(rnd_idx, len(X) // self.batch_size):
                    X_batch, y_batch = X[rnd_indices], y[rnd_indices]
                    feed_dict = {self._X: X_batch, self._y: y_batch}
                    if self._training is not None:
                        feed_dict[self._training] = True
                    sess.run(self._training_op, feed_dict=feed_dict)
                    if extra_update_ops:
                        sess.run(extra_update_ops, feed_dict=feed_dict)
                if X_valid is not None and y_valid is not None:
                    loss_val, acc_val = sess.run([self._loss, self._accuracy],
                                                 feed_dict={self._X: X_valid,
                                                            self._y: y_valid})
                    if loss_val < best_loss:
                        best_params = self._get_model_params()
                        best_loss = loss_val
                        checks_without_progress = 0
                    else:
                        checks_without_progress += 1
                    print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
                        epoch, loss_val, best_loss, acc_val * 100))
                    if checks_without_progress > max_checks_without_progress:
                        print("Early stopping!")
                        break
                else:
                    loss_train, acc_train = sess.run([self._loss, self._accuracy],
                                                     feed_dict={self._X: X_batch,
                                                                self._y: y_batch})
                    print("{}\tLast training batch loss: {:.6f}\tAccuracy: {:.2f}%".format(
                        epoch, loss_train, acc_train * 100))
            # If we used early stopping then rollback to the best model found
            if best_params:
                self._restore_model_params(best_params)
            return self

    def predict_proba(self, X):
        if not self._session:
            raise NotFittedError("This %s instance is not fitted yet" % self.__class__.__name__)
        with self._session.as_default() as sess:
            return self._Y_proba.eval(feed_dict={self._X: X})

    def predict(self, X):
        class_indices = np.argmax(self.predict_proba(X), axis=1)
        return np.array([[self.classes_[class_index]]
                         for class_index in class_indices], np.int32)

    def save(self, path):
        self._saver.save(self._session, path)

param_distribs = {
  "n_neurons": [10, 50, 100, 150],
  "learning_rate": [0.1],
  "n_hidden_layers": [0, 1, 5, 10]
}




In [12]:
rnd_search = RandomizedSearchCV(DNNClassifier(random_state=42), param_distribs, n_iter=10, n_jobs=1,
                                random_state=42, verbose=2)
rnd_search.fit(x_data_train, y_data_train, n_epochs=10, X_valid=x_data_test, y_valid=y_data_test)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] n_neurons=10, n_hidden_layers=0, learning_rate=0.1 ..............
0	Validation loss: 2.843427	Best loss: 2.843427	Accuracy: 26.33%
1	Validation loss: 2.838906	Best loss: 2.838906	Accuracy: 26.87%
2	Validation loss: 2.856728	Best loss: 2.838906	Accuracy: 24.07%
3	Validation loss: 2.867692	Best loss: 2.838906	Accuracy: 25.83%
4	Validation loss: 2.872555	Best loss: 2.838906	Accuracy: 24.59%
5	Validation loss: 2.863784	Best loss: 2.838906	Accuracy: 24.63%
6	Validation loss: 2.888189	Best loss: 2.838906	Accuracy: 25.51%
7	Validation loss: 2.915778	Best loss: 2.838906	Accuracy: 23.80%
8	Validation loss: 2.861585	Best loss: 2.838906	Accuracy: 26.83%
9	Validation loss: 2.831385	Best loss: 2.831385	Accuracy: 27.38%
[CV]  n_neurons=10, n_hidden_layers=0, learning_rate=0.1, total= 1.6min
[CV] n_neurons=10, n_hidden_layers=0, learning_rate=0.1 ..............


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.6min remaining:    0.0s


0	Validation loss: 2.828342	Best loss: 2.828342	Accuracy: 27.79%
1	Validation loss: 2.868824	Best loss: 2.828342	Accuracy: 24.63%
2	Validation loss: 2.880007	Best loss: 2.828342	Accuracy: 23.50%
3	Validation loss: 2.828470	Best loss: 2.828342	Accuracy: 27.36%
4	Validation loss: 2.847561	Best loss: 2.828342	Accuracy: 27.79%
5	Validation loss: 2.899004	Best loss: 2.828342	Accuracy: 25.59%
6	Validation loss: 2.885575	Best loss: 2.828342	Accuracy: 24.50%
7	Validation loss: 2.833725	Best loss: 2.828342	Accuracy: 26.39%
8	Validation loss: 2.844204	Best loss: 2.828342	Accuracy: 26.73%
9	Validation loss: 2.895601	Best loss: 2.828342	Accuracy: 25.65%
[CV]  n_neurons=10, n_hidden_layers=0, learning_rate=0.1, total= 1.6min
[CV] n_neurons=10, n_hidden_layers=0, learning_rate=0.1 ..............
0	Validation loss: 2.836211	Best loss: 2.836211	Accuracy: 26.08%
1	Validation loss: 2.828306	Best loss: 2.828306	Accuracy: 25.04%
2	Validation loss: 2.831997	Best loss: 2.828306	Accuracy: 26.16%
3	Validation

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed: 88.6min finished


0	Validation loss: 2.872181	Best loss: 2.872181	Accuracy: 26.32%
1	Validation loss: 2.886090	Best loss: 2.872181	Accuracy: 25.78%
2	Validation loss: 2.902913	Best loss: 2.872181	Accuracy: 22.85%
3	Validation loss: 2.851068	Best loss: 2.851068	Accuracy: 26.65%
4	Validation loss: 2.893062	Best loss: 2.851068	Accuracy: 24.69%
5	Validation loss: 2.851321	Best loss: 2.851068	Accuracy: 26.26%
6	Validation loss: 2.879363	Best loss: 2.851068	Accuracy: 24.06%
7	Validation loss: 2.846872	Best loss: 2.846872	Accuracy: 25.77%
8	Validation loss: 2.904474	Best loss: 2.846872	Accuracy: 25.01%
9	Validation loss: 2.884920	Best loss: 2.846872	Accuracy: 25.85%


RandomizedSearchCV(cv=None, error_score='raise',
          estimator=DNNClassifier(activation=<function relu at 0x7f2c7f4921e0>,
       batch_norm_momentum=None, batch_size=20, dropout_rate=None,
       initializer=<tensorflow.python.ops.init_ops.VarianceScaling object at 0x7f2c78611e10>,
       learning_rate=0.01, n_hidden_layers=5, n_neurons=100,
       optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
       random_state=42),
          fit_params=None, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'n_neurons': [10, 50, 100, 150], 'learning_rate': [0.1], 'n_hidden_layers': [0, 1, 5, 10]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=2)